<a href="https://colab.research.google.com/github/aaaksenova/DeepPavlov_internship/blob/change/sf_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Скачивание и импорт библиотек

In [1]:
! pip install transformers
! pip install sentence-transformers
! pip install spacy --upgrade
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
     |████████████████████████████████| 636 kB 49.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 85 kB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 24.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=a674de28c2b38a55b56e0b8803d1b6d9a06327cb7af2426027b0d88a092674ad
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
     |████████████████████████████████| 6.4 MB 4.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 456 kB 50.3 MB/s 
     |

In [ ]:
! pip uninstall scikit-learn
! pip install scikit-learn==0.21.2

Found existing installation: scikit-learn 0.21.2
Uninstalling scikit-learn-0.21.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/scikit_learn-0.21.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/sklearn/*
Proceed (y/n)? y
  Successfully uninstalled scikit-learn-0.21.2
  Using cached scikit_learn-0.21.2-cp37-cp37m-manylinux1_x86_64.whl (6.7 MB)


In [2]:
import json
import requests
from tqdm.auto import tqdm
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('brown')
from nltk.corpus import brown
from nltk.tokenize.texttiling import TextTilingTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, BertTokenizer, BertForNextSentencePrediction
import torch 
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


# Подготовка классификатора речевых функций

In [ ]:
# Классификатор

! wget https://raw.githubusercontent.com/aaaksenova/DeepPavlov_internship/change/classifier_fixed.py

--2021-08-11 14:41:56--  https://raw.githubusercontent.com/aaaksenova/DeepPavlov_internship/change/classifier_fixed.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19661 (19K) [text/plain]
Saving to: ‘classifier_fixed.py’

classifier_fixed.py 100%[===================>]  19.20K  --.-KB/s    in 0.001s  

2021-08-11 14:41:56 (17.8 MB/s) - ‘classifier_fixed.py’ saved [19661/19661]



In [3]:
# Размеченные данные

! wget https://raw.githubusercontent.com/lnpetrova/speech_function/main/labeled_data.json

--2021-08-12 10:58:53--  https://raw.githubusercontent.com/lnpetrova/speech_function/main/labeled_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533435 (1.5M) [text/plain]
Saving to: ‘labeled_data.json’

labeled_data.json   100%[===================>]   1.46M  --.-KB/s    in 0.07s   

2021-08-12 10:58:54 (20.7 MB/s) - ‘labeled_data.json’ saved [1533435/1533435]



In [ ]:
! wget https://raw.githubusercontent.com/lnpetrova/speech_function/main/SBC058.json

--2021-08-11 14:41:57--  https://raw.githubusercontent.com/lnpetrova/speech_function/main/SBC058.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46418 (45K) [text/plain]
Saving to: ‘SBC058.json’

SBC058.json         100%[===================>]  45.33K  --.-KB/s    in 0.01s   

2021-08-11 14:41:58 (3.48 MB/s) - ‘SBC058.json’ saved [46418/46418]



In [ ]:
! wget https://raw.githubusercontent.com/lnpetrova/speech_function/main/track_list

--2021-08-11 14:41:58--  https://raw.githubusercontent.com/lnpetrova/speech_function/main/track_list
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14538 (14K) [text/plain]
Saving to: ‘track_list’

track_list          100%[===================>]  14.20K  --.-KB/s    in 0s      

2021-08-11 14:41:58 (69.6 MB/s) - ‘track_list’ saved [14538/14538]



In [ ]:
! wget -O nn_classifier.pickle https://github.com/lnpetrova/speech_function/blob/main/nn_classifier.pickle?raw=true

--2021-08-11 14:41:58--  https://github.com/lnpetrova/speech_function/blob/main/nn_classifier.pickle?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/lnpetrova/speech_function/raw/main/nn_classifier.pickle [following]
--2021-08-11 14:41:59--  https://github.com/lnpetrova/speech_function/raw/main/nn_classifier.pickle
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lnpetrova/speech_function/main/nn_classifier.pickle [following]
--2021-08-11 14:41:59--  https://raw.githubusercontent.com/lnpetrova/speech_function/main/nn_classifier.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [ ]:
! wget -O scaler.pickle https://github.com/lnpetrova/speech_function/blob/main/scaler.pickle?raw=true

--2021-08-11 14:42:00--  https://github.com/lnpetrova/speech_function/blob/main/scaler.pickle?raw=true
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/lnpetrova/speech_function/raw/main/scaler.pickle [following]
--2021-08-11 14:42:00--  https://github.com/lnpetrova/speech_function/raw/main/scaler.pickle
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lnpetrova/speech_function/main/scaler.pickle [following]
--2021-08-11 14:42:00--  https://raw.githubusercontent.com/lnpetrova/speech_function/main/scaler.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent

In [ ]:
%run classifier_fixed.py

# sklearn 0.21.2

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

# Ground truth

Сейчас здесь парсится диалог из Санта Барбары. Чтобы вседальше работало нужно из вашего файла в SPEAKER положить спикеров, в REPLICS реплики, в TRUE тэги речевых функций

In [4]:
with open('/content/labeled_data.json') as fh:
    tagged = json.load(fh)

tag_dial = []

for i in tagged[0]['completions'][0]['result']:
    if 'taxonomy' in i['from_name']:
        tag_dial.append((i['value']['text'], i['from_name'].strip('taxonomy')))

In [5]:
SPEAKER = [i['speaker'] for i in tagged[0]['data']['text']]
REPLICS = [i[0].replace('\n', ' ').replace('PAUSE', ' ') for i in tag_dial]
TRUE = ['Open' if 'Ope' in i[1] else 'Same' for i in tag_dial]

In [ ]:
pred = []
for i in classify_labels(REPLICS, SPEAKER):
    if 'Open' in i:
        pred.append('Open')
    else:
        pred.append('Same')
print(classification_report(TRUE, pred[1:]))

              precision    recall  f1-score   support

        Open       0.56      0.71      0.63        21
        Same       0.99      0.97      0.98       470

    accuracy                           0.96       491
   macro avg       0.77      0.84      0.80       491
weighted avg       0.97      0.96      0.97       491



# Tf-Idf

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(REPLICS).toarray()

In [ ]:
pred = []
idx = [0.0002, 0.003, 0.004, 0.005, 0.006, 0.1, 0.2]


for i in tqdm(idx):
    pred = []
    for s in range(len(X))[2:]:
        sent_0 = X[s-2]
        sent_1 = X[s-1]
        cont_right = X[s]
        cont_left = sent_0 + sent_1 
        vote_tfidf = abs(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0]) < i
        #print(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0], vote_bert)
        if vote_tfidf:
            pred.append('Open')
        else:
            pred.append('Same')
    print('Classification report for cosine similarity', i)
    print(classification_report(TRUE[2:], pred))

Classification report for cosine similarity 0.0002
              precision    recall  f1-score   support

        Open       0.04      0.55      0.08        20
        Same       0.96      0.49      0.65       469

    accuracy                           0.49       489
   macro avg       0.50      0.52      0.37       489
weighted avg       0.92      0.49      0.63       489

Classification report for cosine similarity 0.003
              precision    recall  f1-score   support

        Open       0.04      0.55      0.08        20
        Same       0.96      0.49      0.65       469

    accuracy                           0.49       489
   macro avg       0.50      0.52      0.37       489
weighted avg       0.92      0.49      0.63       489

Classification report for cosine similarity 0.004
              precision    recall  f1-score   support

        Open       0.04      0.55      0.08        20
        Same       0.96      0.49      0.65       469

    accuracy                   

# TextTilingTokenizer

In [ ]:
text = '\n\n'.join(REPLICS)

In [ ]:
tt = TextTilingTokenizer(w=2, k=3)
t = tt.tokenize(text)

In [ ]:
pred = []
for i in t:
    turn = ['Open'] + ['Same'] * (len(i.strip('\n\n').split('\n\n')) - 1) 
    pred.extend(turn)

In [ ]:
print(classification_report(TRUE, pred))

              precision    recall  f1-score   support

        Open       0.19      0.03      0.06       120
        Same       0.75      0.95      0.84       371

    accuracy                           0.73       491
   macro avg       0.47      0.49      0.45       491
weighted avg       0.62      0.73      0.65       491



# Cosine similarity experiments

## TODBERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TODBERT/TOD-BERT-JNT-V1")
model = AutoModelForMaskedLM.from_pretrained("TODBERT/TOD-BERT-JNT-V1")

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings

def BERT(sent, model, tokenizer):
    model.to('cuda')
    encoded_input = tokenizer(sent, padding=True, truncation=True, max_length=128, return_tensors='pt').to('cuda')
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embedding.cpu().detach().numpy()[0]

In [ ]:
pred = []
idx = [ 0.96, 0.97, 0.98, 0.99, 0.995, 0.998]


for i in tqdm(idx):
    pred = []
    for s in range(len(REPLICS))[2:]:
        sent_0 = REPLICS[s-2]
        sent_1 = REPLICS[s-1]
        sent_2 = REPLICS[s]
        cont_right = BERT(sent_2, model, tokenizer)
        cont_left = BERT(sent_0 + sent_1, model, tokenizer)  
        vote_bert = abs(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0]) < i
        #print(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0], vote_bert)
        if vote_bert:
            pred.append('Open')
        else:
            pred.append('Same')
    print('Classification report for cosine similarity', i)
    print(classification_report(TRUE[2:], pred))

Classification report for cosine similarity 0.96
              precision    recall  f1-score   support

        Open       0.00      0.00      0.00        20
        Same       0.96      1.00      0.98       469

    accuracy                           0.96       489
   macro avg       0.48      0.50      0.49       489
weighted avg       0.92      0.96      0.94       489



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for cosine similarity 0.97
              precision    recall  f1-score   support

        Open       0.00      0.00      0.00        20
        Same       0.96      1.00      0.98       469

    accuracy                           0.96       489
   macro avg       0.48      0.50      0.49       489
weighted avg       0.92      0.96      0.94       489



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for cosine similarity 0.98
              precision    recall  f1-score   support

        Open       0.00      0.00      0.00        20
        Same       0.96      1.00      0.98       469

    accuracy                           0.96       489
   macro avg       0.48      0.50      0.49       489
weighted avg       0.92      0.96      0.94       489



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Classification report for cosine similarity 0.99
              precision    recall  f1-score   support

        Open       0.04      0.15      0.06        20
        Same       0.96      0.85      0.90       469

    accuracy                           0.82       489
   macro avg       0.50      0.50      0.48       489
weighted avg       0.92      0.82      0.87       489

Classification report for cosine similarity 0.995
              precision    recall  f1-score   support

        Open       0.04      0.85      0.08        20
        Same       0.96      0.16      0.27       469

    accuracy                           0.18       489
   macro avg       0.50      0.50      0.17       489
weighted avg       0.92      0.18      0.26       489

Classification report for cosine similarity 0.998
              precision    recall  f1-score   support

        Open       0.04      0.95      0.08        20
        Same       0.89      0.02      0.03       469

    accuracy                     

## DeepPavlov BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/bert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/bert-base-cased-conversational")

In [ ]:
pred = []
idx = [0.7, 0.8, 0.9]


for i in tqdm(idx):
    pred = []
    for s in range(len(REPLICS))[2:]:
        sent_0 = REPLICS[s-2]
        sent_1 = REPLICS[s-1]
        sent_2 = REPLICS[s]
        cont_right = BERT(sent_2, model, tokenizer)
        cont_left = BERT(sent_0 + sent_1, model, tokenizer)  
        vote_bert = abs(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0]) < i
        #print(cosine_similarity(cont_left.reshape(1, -1), cont_right.reshape(1, -1))[0][0], vote_bert)
        if vote_bert:
            pred.append('Open')
        else:
            pred.append('Same')
    print('Classification report for cosine similarity', i)
    print(classification_report(TRUE[2:], pred))

Classification report for cosine similarity 0.7
              precision    recall  f1-score   support

        Open       0.04      0.55      0.07        20
        Same       0.95      0.37      0.53       469

    accuracy                           0.38       489
   macro avg       0.49      0.46      0.30       489
weighted avg       0.91      0.38      0.51       489

Classification report for cosine similarity 0.8
              precision    recall  f1-score   support

        Open       0.04      0.90      0.08        20
        Same       0.95      0.07      0.14       469

    accuracy                           0.11       489
   macro avg       0.49      0.49      0.11       489
weighted avg       0.91      0.11      0.14       489

Classification report for cosine similarity 0.9
              precision    recall  f1-score   support

        Open       0.04      0.90      0.07        20
        Same       0.86      0.03      0.05       469

    accuracy                          

## BERT Next Sentence Prediction

## Logit Difference

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pred = []
logs = np.arange(1, 2.7, 0.3)


for i in logs:
    pred = []
    for s in range(len(REPLICS))[2:]:
        sent_0 = REPLICS[s-2]
        sent_1 = REPLICS[s-1]
        prompt = sent_0 + sent_1
        next_sentence = REPLICS[s]
        encoding = tokenizer(prompt, next_sentence, return_tensors='pt')
        outputs = model(**encoding, labels=torch.LongTensor([1]))
        logits = outputs.logits
        if abs(abs(logits[0, 0]) - abs(logits[0, 1])) > i:
            pred.append('Open')
        else:
            pred.append('Same')
    print('Classification report for logit difference', i)
    print(classification_report(TRUE[2:], pred))

Classification report for logit difference 1.0
              precision    recall  f1-score   support

        Open       0.05      0.45      0.10        20
        Same       0.97      0.67      0.79       469

    accuracy                           0.66       489
   macro avg       0.51      0.56      0.44       489
weighted avg       0.93      0.66      0.76       489

Classification report for logit difference 1.3
              precision    recall  f1-score   support

        Open       0.06      0.25      0.10        20
        Same       0.96      0.83      0.89       469

    accuracy                           0.81       489
   macro avg       0.51      0.54      0.49       489
weighted avg       0.93      0.81      0.86       489

Classification report for logit difference 1.6
              precision    recall  f1-score   support

        Open       0.02      0.05      0.03        20
        Same       0.96      0.89      0.92       469

    accuracy                           0.

## Next VS Not Next

In [ ]:
pred = []
for s in range(len(REPLICS))[2:]:
    sent_0 = REPLICS[s-2]
    sent_1 = REPLICS[s-1]
    prompt = sent_0 + sent_1
    next_sentence = REPLICS[s]
    encoding = tokenizer(prompt, next_sentence, return_tensors='pt')
    outputs = model(**encoding, labels=torch.LongTensor([1]))
    logits = outputs.logits
    if logits[0, 0] < logits[0, 1]:
        pred.append('Open')
    else:
        pred.append('Same')

In [ ]:
print(classification_report(TRUE[2:], pred))

              precision    recall  f1-score   support

        Open       0.02      0.05      0.03        20
        Same       0.96      0.89      0.92       469

    accuracy                           0.86       489
   macro avg       0.49      0.47      0.48       489
weighted avg       0.92      0.86      0.89       489



## LogReg based on BERT embeddings

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TODBERT/TOD-BERT-JNT-V1")
model = AutoModelForMaskedLM.from_pretrained("TODBERT/TOD-BERT-JNT-V1")

In [ ]:
vecs = []
for s in range(len(REPLICS))[2:]:
    sent_0 = REPLICS[s-2]
    sent_1 = REPLICS[s-1]
    sent_2 = REPLICS[s]
    cont_left = BERT(sent_0 + sent_1, model, tokenizer)
    cont_right = BERT(sent_2, model, tokenizer)
    vecs.append(np.concatenate((cont_left, cont_right), axis=None))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vecs, TRUE[2:], test_size=0.3, random_state=42, stratify=TRUE[2:])

In [ ]:
clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train, y_train)
pred = clf.predict(X_test)
print(classification_report(y_test, pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

        Open       0.00      0.00      0.00         6
        Same       0.96      0.96      0.96       141

    accuracy                           0.93       147
   macro avg       0.48      0.48      0.48       147
weighted avg       0.92      0.93      0.92       147



# MRDA

In [ ]:
! wget https://raw.githubusercontent.com/NathanDuran/MRDA-Corpus/master/mrda_data/full_set.txt

--2021-08-11 15:12:33--  https://raw.githubusercontent.com/NathanDuran/MRDA-Corpus/master/mrda_data/full_set.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5201358 (5.0M) [text/plain]
Saving to: ‘full_set.txt’

full_set.txt        100%[===================>]   4.96M  24.9MB/s    in 0.2s    

2021-08-11 15:12:34 (24.9 MB/s) - ‘full_set.txt’ saved [5201358/5201358]



In [ ]:
with open('full_set.txt') as fh:
    sent = [i.strip() for i in fh.readlines()]

In [ ]:
# Будем считать, что такие типы реплик соответствуют Open
# Про MRDA читать тут https://github.com/NathanDuran/MRDA-Corpus/blob/master/mrda_manual.pdf

replics = []
for s in sent:
    if s.split('|')[3] in ['qy', 'qw', 'qrr', 'qo']:
        replics.append((s.split('|')[0], s.split('|')[1], 'fake_open'))
    else:
        replics.append((s.split('|')[0], s.split('|')[1], 'not_open'))

In [ ]:
vecs =  [BERT(i[1], model, tokenizer) for i in tqdm(replics[:10000])]
true = [i[2] for i in replics[:10000]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vecs, true, test_size=0.3, random_state=42, stratify=true)

In [ ]:
# Обучим простенький классификатор на MRDA

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
pred = clf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   fake_open       0.76      0.76      0.76       227
    not_open       0.98      0.98      0.98      2773

    accuracy                           0.96      3000
   macro avg       0.87      0.87      0.87      3000
weighted avg       0.96      0.96      0.96      3000



In [ ]:
vecs = []
for s in range(len(REPLICS))[2:]:
    sent_0 = REPLICS[s-2]
    sent_1 = REPLICS[s-1]
    sent_2 = REPLICS[s]
    cont_left = BERT(sent_0 + sent_1, model, tokenizer)
    cont_right = BERT(sent_2, model, tokenizer)
    vecs.append(np.concatenate((cont_left, cont_right), axis=None))
    print(sent_2 + '\t\t' + TRUE[s] + '\t\t' + clf.predict(cont_right.reshape(1, -1))[0])

 I know.		Same		not_open
 And it's hard habit to break.		Same		not_open
 Usually I don't...		Same		not_open
 It is.		Same		not_open
 ʔuh Usually I don't stay up late.		Same		not_open
 But it's like
 if I'm up after midnight ?
 It's just like.		Same		not_open
 Hm.
 Yeah yeah.		Same		not_open
What can I do now.		Same		not_open
PAUSE		Same		not_open
PAUSE		Same		not_open
 I still can't ʔuh...		Open		not_open
God I still can't believe Tim bitching around and
 he lied too.		Open		not_open
 He said that he talked to Ron		Same		not_open
About what.		Same		not_open
 About ʔuh the way they were feeling
 of them being the only ones cleaning the house
 and all this other shit ?		Same		fake_open
I mean what they don't realize
 is like
 shit
 when Ron gets home from work		Same		not_open
 Yeah.		Same		not_open
 Unlike Tim
 he has to work
 for every little dime that he makes.		Same		not_open
You know ?		Same		fake_open
 Yeah.		Same		not_open
He doesn't get any breaks.		Same		not_open
Yeahʔ		Same		not

In [ ]:
pred =  clf.predict([BERT(i, model, tokenizer) for i in tqdm(REPLICS)])
pred = ['Open' if 'open' in i else 'Same' for i in pred]

In [ ]:
print(classification_report(TRUE, pred))

              precision    recall  f1-score   support

        Open       0.04      1.00      0.08        21
        Same       0.00      0.00      0.00       470

    accuracy                           0.04       491
   macro avg       0.02      0.50      0.04       491
weighted avg       0.00      0.04      0.00       491



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## CoBot Conversation Evaluator

In [ ]:
API_KEY = 'QFPxaMUoPi5qcax2FBt9D6Y6vAgLRBbn56TW1iO3'

headers = {'Content-Type': 'application/json;charset=utf-8', 
           'x-api-key': API_KEY}
SERVICE_URL = 'https://pssen6zq8k.execute-api.us-east-1.amazonaws.com/prod//conversationevaluation/v1/batchConversationEvaluation'

conversation = {
"currentUtterance": 'I love NLP!',
"currentResponse": 'Me too!',
"pastUtterances": [],
"pastResponses": []
}

requests.request(url=SERVICE_URL, 
                 headers=headers, 
                 data=json.dumps({'conversations': [conversation]}), 
                 method='POST').json()

{'conversationEvaluationScores': [{'isResponseComprehensible': 0.517,
   'isResponseErroneous': 0.579,
   'isResponseInteresting': 0.211,
   'isResponseOnTopic': 0.374,
   'responseEngagesUser': 0.566}],
 'requestId': 'cad555be-5c73-4102-bcff-4ba4fd02ddda',
 'version': '2.0'}

In [ ]:
full_conv = []

for i in range(len(REPLICS[:-1])):
    full_conv.append({
    "currentUtterance": REPLICS[i],
    "currentResponse": REPLICS[i+1],
    "pastUtterances": [],
    "pastResponses": []
    })

In [ ]:
tags = []

for conv in tqdm(full_conv):
    tags.append(requests.request(url=SERVICE_URL, 
                 headers=headers, 
                 data=json.dumps({'conversations': [conv]}), 
                 method='POST').json()['conversationEvaluationScores'][0]['isResponseOnTopic'])

In [ ]:
data

<_io.TextIOWrapper name='labeled_data.json' mode='r' encoding='UTF-8'>

In [ ]:
full_conv[398]

{'currentResponse': " Cause I've been just  craving seafood.",
 'currentUtterance': ' Yeah.',
 'pastResponses': [],
 'pastUtterances': []}

In [ ]:
tags = [0] + tags

In [ ]:
idx = [0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33]

for i in idx:
    pred = ['Open' if tag < i else 'Same' for tag in tags]
    print('Classification report for Response on topic less then', i)
    print(classification_report(TRUE, pred))

Classification report for Response on topic less then 0.2
              precision    recall  f1-score   support

        Open       0.03      0.29      0.06        21
        Same       0.95      0.62      0.75       470

    accuracy                           0.60       491
   macro avg       0.49      0.45      0.40       491
weighted avg       0.91      0.60      0.72       491

Classification report for Response on topic less then 0.21
              precision    recall  f1-score   support

        Open       0.03      0.33      0.06        21
        Same       0.95      0.58      0.72       470

    accuracy                           0.57       491
   macro avg       0.49      0.46      0.39       491
weighted avg       0.91      0.57      0.69       491

Classification report for Response on topic less then 0.22
              precision    recall  f1-score   support

        Open       0.03      0.33      0.06        21
        Same       0.95      0.55      0.70       470

    ac

## CoBot Topic Classifier

In [ ]:
headers = {'Content-Type': 'application/json;charset=utf-8', 
           'x-api-key': API_KEY}
SERVICE_URL = 'https://gpri4649wf.execute-api.us-east-1.amazonaws.com/prod/topic/v1/batchTopics'


user_sentences = ['do you like muslims?',
                                   'what are your thoughts about donald trump?']

requests.request(url=f'{SERVICE_URL}',
                                  headers=headers,
                                  data=json.dumps({'utterances': user_sentences}),
                                  method='POST',
                                  timeout=1).json()

{'requestId': 'c2e737ab-882b-4095-90bd-1431c7adb3f9',
 'topics': [{'confidence': 0.816,
   'text': 'do you like muslims?',
   'topicClass': 'Religion',
   'topicKeywords': [{'confidence': 0.897, 'keyword': 'Muslims'}]},
  {'confidence': 999.0,
   'text': 'what are your thoughts about donald trump?',
   'topicClass': 'Politics',
   'topicKeywords': [{'confidence': 999.0, 'keyword': 'donald trump'}]}],
 'version': '2.0'}

In [ ]:
import time
tags = []

for rep in tqdm(REPLICS):
    time.sleep(3)
    tags.append(requests.request(url=f'{SERVICE_URL}',
                                  headers=headers,
                                  data=json.dumps({'utterances': [rep]}),
                                  method='POST',
                                  timeout=2).json())

In [ ]:
tags = [i['topics'][0]['topicClass'] for i in tags]

In [ ]:
pred = []

for i in range(len(tags[1:])):
    if tags[i-1] != tags[i]:
        pred.append('Open')
    else:
        pred.append('Same')

In [ ]:
pred = ['Open'] + pred

In [ ]:
print(classification_report(TRUE, pred))

              precision    recall  f1-score   support

        Open       0.04      0.76      0.08        21
        Same       0.96      0.24      0.38       470

    accuracy                           0.26       491
   macro avg       0.50      0.50      0.23       491
weighted avg       0.92      0.26      0.37       491



In [ ]:
for i, j in zip(REPLICS, tags):
    print(i + '\t\t' + j)

 God  I said I wasn't gonna do this anymore.		Religion
 Stay up late.  Kinda defeats the purpose of getting up in the morning.		Other
 I know.		Phatic
 And it's hard habit to break.		Psychology
 Usually I don't...		Phatic
 It is.		Phatic
 ʔuh Usually I don't stay up late.		Celebrities
 But it's like  if I'm up after midnight ?  It's just like.		Weather_Time
 Hm.  Yeah yeah.		Phatic
What can I do now.		Phatic
 		Other
 		Other
 I still can't ʔuh...		Phatic
God I still can't believe Tim bitching around and  he lied too.		Sex_Profanity
 He said that he talked to Ron		Other
About what.		Phatic
 About ʔuh the way they were feeling  of them being the only ones cleaning the house  and all this other shit ?		Sex_Profanity
I mean what they don't realize  is like  shit  when Ron gets home from work		Sex_Profanity
 Yeah.		Phatic
 Unlike Tim  he has to work  for every little dime that he makes.		Other
You know ?		Phatic
 Yeah.		Phatic
He doesn't get any breaks.		Travel_Geo
Yeahʔ		Phatic
 Tim is on

## CoBot Dialog Actions Classifier

In [6]:
API_KEY = 'QFPxaMUoPi5qcax2FBt9D6Y6vAgLRBbn56TW1iO3'

headers = {'Content-Type': 'application/json;charset=utf-8', 
           'x-api-key': API_KEY}
SERVICE_URL = 'https://8v6uiqpymj.execute-api.us-east-1.amazonaws.com/prod/'

conversation = {
"currentUtterance": 'I love NLP!',
"currentResponse": 'Me too!',
"pastUtterances": [],
"pastResponses": []
}

requests.request(url=SERVICE_URL, 
                 headers=headers, 
                 data=json.dumps({'conversations': [conversation]}), 
                 method='POST').json()

{'dialogActIntents': [{'dialogActIntent': 'General_ChatIntent',
   'topic': 'Phatic'}],
 'version': '2.1'}

In [16]:
full_conv = []

for i in range(len(REPLICS[:-1])):
    curu = REPLICS[i]
    curr = REPLICS[i+1]
    if REPLICS[i] == ' ':
        curu = 'k'
    if REPLICS[i+1] == ' ':
        corr = 'k'
    full_conv.append({
    "currentUtterance": curu,
    "currentResponse": curr,
    "pastUtterances": [],
    "pastResponses": []
    })

In [17]:
tags = []

for i in tqdm(range(len(full_conv))):
    try:
        tags.append(requests.request(url=SERVICE_URL, 
                 headers=headers, 
                 data=json.dumps({'conversations': [full_conv[i]]}), 
                 method='POST').json()['dialogActIntents'][0]['topic'])
    except:
        tags.append(requests.request(url=SERVICE_URL, 
                 headers=headers, 
                 data=json.dumps({'conversations': [full_conv[i-1]]}), 
                 method='POST').json()['dialogActIntents'][0]['topic'])

In [23]:
len(tags)

490

In [24]:
pred = []

for i in range(len(tags[1:])):       
    if tags[i-1] != tags[i]:
        pred.append('Open')
    else:
        pred.append('Same')

In [25]:
pred = ['Open', 'Same'] + pred

In [26]:
print(classification_report(TRUE, pred))

              precision    recall  f1-score   support

        Open       0.03      0.52      0.06        21
        Same       0.93      0.30      0.46       470

    accuracy                           0.31       491
   macro avg       0.48      0.41      0.26       491
weighted avg       0.90      0.31      0.44       491



In [27]:
for i, j in zip(REPLICS[1:], tags):
    print(i + '\t\t' + j)

 Stay up late.  Kinda defeats the purpose of getting up in the morning.		Phatic
 I know.		Entertainment_General
 And it's hard habit to break.		Phatic
 Usually I don't...		Other
 It is.		Phatic
 ʔuh Usually I don't stay up late.		Phatic
 But it's like  if I'm up after midnight ?  It's just like.		Phatic
 Hm.  Yeah yeah.		Phatic
What can I do now.		Phatic
 		Other
 		Inappropriate_Content
 I still can't ʔuh...		Inappropriate_Content
God I still can't believe Tim bitching around and  he lied too.		Phatic
 He said that he talked to Ron		Phatic
About what.		Entertainment_Books
 About ʔuh the way they were feeling  of them being the only ones cleaning the house  and all this other shit ?		Other
I mean what they don't realize  is like  shit  when Ron gets home from work		Inappropriate_Content
 Yeah.		Inappropriate_Content
 Unlike Tim  he has to work  for every little dime that he makes.		Phatic
You know ?		Entertainment_Movies
 Yeah.		Entertainment_Movies
He doesn't get any breaks.		Phatic
Y

# Dialog Parsing

In [ ]:
d = requests.get('https://files.deeppavlov.ai/oleg/deepy-dream/0.json'.format(i)).json()

In [ ]:
dialogs = []
for i in tqdm(range(0, 7001, 1000)):
  d = requests.get('https://files.deeppavlov.ai/oleg/deepy-dream/{}.json'.format(i)).json()
  dialogs.extend(d)

In [ ]:
clean_dialogs = []
for dial in dialogs:
    reps = []
    if len(dial['utterances']) >= 8:
        for i in dial['utterances']:
            s_list = nltk.tokenize.sent_tokenize(i['text'])
            for j in s_list:
                reps.append({i['user']['user_type'] : j})
    if reps:
        clean_dialogs.append(reps)

In [ ]:
with open('reps.json', "w") as write_file:
    json.dump(clean_dialogs, write_file)